In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('../inputs/ready_data/preprocessed_all_matches.csv')
df

,Division,Date,HomeTeam,AwayTeam,Full_Time_Result,Half_Time_Home_Team_Goals,Half_Time_Away_Team_Goals,Half_Time_Result,Home_Team_Shots,Away_Team_Shots,...,HOME_TRUESKILL_MU_NO_RESET,AWAY_TRUESKILL_MU_NO_RESET,HOME_TRUESKILL_SIGMA_NO_RESET,AWAY_TRUESKILL_SIGMA_NO_RESET,DRAW_CHANCE_NO_RESET,HOME_TRUESKILL_MU_SEASON,AWAY_TRUESKILL_MU_SEASON,HOME_TRUESKILL_SIGMA_SEASON,AWAY_TRUESKILL_SIGMA_SEASON,DRAW_CHANCE_SEASON
0,SP1,2006-08-26,Valencia,Betis,1,1.0,0.0,H,14.0,8.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
1,SP1,2006-08-27,Ath Bilbao,Sociedad,0,1.0,0.0,H,7.0,10.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
2,SP1,2006-08-27,Espanol,Gimnastic,2,0.0,0.0,D,20.0,18.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
3,SP1,2006-08-27,La Coruna,Zaragoza,1,2.0,1.0,H,14.0,13.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
4,SP1,2006-08-27,Osasuna,Getafe,2,0.0,0.0,D,15.0,7.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,SP1,2020-08-03,Valladolid,Ath Bilbao,2,0.0,2.0,A,19.0,10.0,...,24.259624,25.818350,0.756801,0.752533,0.951216,24.348685,24.163046,1.496884,1.480508,0.941283
5206,SP1,2020-08-03,Levante,Granada,0,1.0,0.0,H,11.0,10.0,...,24.489846,23.545186,0.769946,0.779812,0.971007,23.581250,24.937909,1.598470,1.607929,0.911998
5207,SP1,2020-08-03,Villarreal,Leganes,2,1.0,0.0,H,15.0,12.0,...,26.047712,24.074685,0.760933,0.788714,0.931289,24.829939,22.458682,1.542022,1.587451,0.872025
5208,SP1,2020-08-03,Betis,Real Madrid,1,1.0,1.0,D,14.0,13.0,...,24.989093,29.273576,0.759584,0.784491,0.761516,24.068836,29.788006,1.512679,1.587302,0.619674


In [39]:
elo_and_wins = df
elo_and_wins['ELO_Result'] = 0
elo_and_wins.loc[elo_and_wins['AWAY_TRUESKILL_MU_SEASON'] < elo_and_wins['HOME_TRUESKILL_MU_SEASON'],'ELO_Result'] = 1
elo_and_wins.loc[elo_and_wins['AWAY_TRUESKILL_MU_SEASON'] > elo_and_wins['HOME_TRUESKILL_MU_SEASON'],'ELO_Result'] = 2
#elo_and_wins.loc[elo_and_wins['DRAW_CHANCE_SEASON'] > 0.95,'ELO_Result'] = 0
#elo_and_wins = elo_and_wins.query('Full_Time_Result != 0')
accuracy_score(elo_and_wins['ELO_Result'], elo_and_wins['Full_Time_Result'])

0.47140115163147794

In [46]:
elo_and_wins = df
elo_and_wins['ELO_Result'] = 0
elo_and_wins.loc[elo_and_wins['AWAY_TRUESKILL_MU_NO_RESET'] < elo_and_wins['HOME_TRUESKILL_MU_NO_RESET']+2,'ELO_Result'] = 1
elo_and_wins.loc[elo_and_wins['AWAY_TRUESKILL_MU_NO_RESET'] > elo_and_wins['HOME_TRUESKILL_MU_NO_RESET']+2,'ELO_Result'] = 2
#elo_and_wins.loc[elo_and_wins['DRAW_CHANCE_NO_RESET'] > 0.95,'ELO_Result'] = 0
#elo_and_wins = elo_and_wins.query('Full_Time_Result != 0')
accuracy_score(elo_and_wins['ELO_Result'], elo_and_wins['Full_Time_Result'])

0.5261036468330135

In [45]:
elo_and_wins = df
elo_and_wins['ELO_Result'] = 0
elo_and_wins.loc[(elo_and_wins['AWAY_TRUESKILL_MU_NO_RESET'] < elo_and_wins['HOME_TRUESKILL_MU_NO_RESET']+2) & (elo_and_wins['AWAY_TRUESKILL_MU_SEASON'] < elo_and_wins['HOME_TRUESKILL_MU_SEASON']+2),'ELO_Result'] = 1
elo_and_wins.loc[(elo_and_wins['AWAY_TRUESKILL_MU_NO_RESET'] > elo_and_wins['HOME_TRUESKILL_MU_NO_RESET']+2) & (elo_and_wins['AWAY_TRUESKILL_MU_SEASON'] > elo_and_wins['HOME_TRUESKILL_MU_SEASON']+2),'ELO_Result'] = 2
elo_and_wins.loc[(elo_and_wins['DRAW_CHANCE_NO_RESET'] > 0.47) & (elo_and_wins['DRAW_CHANCE_NO_RESET'] < 0.50),'ELO_Result'] = 0
#elo_and_wins = elo_and_wins.query('Full_Time_Result != 0')
accuracy_score(elo_and_wins['ELO_Result'], elo_and_wins['Full_Time_Result'])

0.5023032629558541

In [48]:
elo_and_wins = df
elo_and_wins['ELO_Result'] = 0
elo_and_wins.loc[elo_and_wins['AWAY_TRUESKILL_MU_NO_RESET'] < elo_and_wins['HOME_TRUESKILL_MU_NO_RESET']+2,'ELO_Result'] = 1
elo_and_wins.loc[elo_and_wins['AWAY_TRUESKILL_MU_NO_RESET'] > elo_and_wins['HOME_TRUESKILL_MU_NO_RESET']+2,'ELO_Result'] = 2
#elo_and_wins.loc[elo_and_wins['DRAW_CHANCE_NO_RESET'] > 0.95,'ELO_Result'] = 0
#elo_and_wins = elo_and_wins.query('Full_Time_Result != 0')
accuracy_score(elo_and_wins['ELO_Result'], elo_and_wins['Full_Time_Result'])
profit_df = pd.DataFrame()

Index(['Division', 'Date', 'HomeTeam', 'AwayTeam', 'Full_Time_Result',
       'Half_Time_Home_Team_Goals', 'Half_Time_Away_Team_Goals',
       'Half_Time_Result', 'Home_Team_Shots', 'Away_Team_Shots',
       'Home_Team_Target_Shots', 'Away_Team_Target_Shots',
       'Home_Team_Fouls_Commited', 'Away_Team_Fouls_Commited',
       'Home_Team_Corners', 'Away_Team_Corners', 'Home_Team_Yellow_Cards',
       'Away_Team_Yellow_Cards', 'Home_Team_Red_Cards',
       'Home_Team_Red_Cards.1', 'Bet365_Home_Win_Odds', 'Bet365_Draw_Odds',
       'Bet365_Away_Win_Odds', 'BetAndWin_Home_Win_Odds',
       'BetAndWin_Draw_Odds', 'BetAndWin_Away_Win_Odds',
       'Interwetten_Home_Win_Odds', 'Interwetten_Draw_Odds',
       'Interwetten_Away_Win_Odds', 'WilliamHill_Home_Win_Odds',
       'WilliamHill_Draw_Odds', 'WilliamHill_Away_Win_Odds',
       'VCBet_Home_Win_Odds', 'VCBet_Draw_Odds', 'VCBet_Away_Win_Odds',
       'Season', 'Home Overall Score', 'Home Attack Score',
       'Home Middle Score', 'Home De

# Veces ganadas por equipo en casa

In [56]:
home_wins = df[['HomeTeam', 'Full_Time_Result', 'Season']]
home_wins.groupby(['HomeTeam', 'Full_Time_Result', 'Season']).size().reset_index()

,HomeTeam,Full_Time_Result,Season,0
0,Alaves,0,2016_2017,8
1,Alaves,0,2017_2018,1
2,Alaves,0,2018_2019,8
3,Alaves,0,2019_2020,5
4,Alaves,1,2016_2017,7
...,...,...,...,...
823,Zaragoza,2,2007_2008,3
824,Zaragoza,2,2009_2010,6
825,Zaragoza,2,2010_2011,7
826,Zaragoza,2,2011_2012,7


In [4]:
import pandas as pd
df = pd.read_csv('../inputs/ready_data/preprocessed_all_matches.csv')
df

,Division,Date,HomeTeam,AwayTeam,Full_Time_Result,Half_Time_Home_Team_Goals,Half_Time_Away_Team_Goals,Half_Time_Result,Home_Team_Shots,Away_Team_Shots,...,HOME_TRUESKILL_MU_NO_RESET,AWAY_TRUESKILL_MU_NO_RESET,HOME_TRUESKILL_SIGMA_NO_RESET,AWAY_TRUESKILL_SIGMA_NO_RESET,DRAW_CHANCE_NO_RESET,HOME_TRUESKILL_MU_SEASON,AWAY_TRUESKILL_MU_SEASON,HOME_TRUESKILL_SIGMA_SEASON,AWAY_TRUESKILL_SIGMA_SEASON,DRAW_CHANCE_SEASON
0,SP1,2006-08-26,Valencia,Betis,1,1.0,0.0,H,14.0,8.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
1,SP1,2006-08-27,Ath Bilbao,Sociedad,0,1.0,0.0,H,7.0,10.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
2,SP1,2006-08-27,Espanol,Gimnastic,2,0.0,0.0,D,20.0,18.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
3,SP1,2006-08-27,La Coruna,Zaragoza,1,2.0,1.0,H,14.0,13.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
4,SP1,2006-08-27,Osasuna,Getafe,2,0.0,0.0,D,15.0,7.0,...,25.000000,25.000000,8.333333,8.333333,0.447214,25.000000,25.000000,8.333333,8.333333,0.447214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,SP1,2020-08-03,Valladolid,Ath Bilbao,2,0.0,2.0,A,19.0,10.0,...,24.259624,25.818350,0.756801,0.752533,0.951216,24.348685,24.163046,1.496884,1.480508,0.941283
5206,SP1,2020-08-03,Levante,Granada,0,1.0,0.0,H,11.0,10.0,...,24.489846,23.545186,0.769946,0.779812,0.971007,23.581250,24.937909,1.598470,1.607929,0.911998
5207,SP1,2020-08-03,Villarreal,Leganes,2,1.0,0.0,H,15.0,12.0,...,26.047712,24.074685,0.760933,0.788714,0.931289,24.829939,22.458682,1.542022,1.587451,0.872025
5208,SP1,2020-08-03,Betis,Real Madrid,1,1.0,1.0,D,14.0,13.0,...,24.989093,29.273576,0.759584,0.784491,0.761516,24.068836,29.788006,1.512679,1.587302,0.619674


In [5]:
elo_list = []
date_list = []
team_list = []
for _, row in df.iterrows():
    home_team = row.HomeTeam
    away_team = row.AwayTeam
    home_elo = row.HOME_TRUESKILL_MU_SEASON
    away_elo = row.AWAY_TRUESKILL_MU_SEASON
    match_date = row.Date
    date_list.append(match_date)
    date_list.append(match_date)
    
    team_list.append(home_team)
    team_list.append(away_team)

    elo_list.append(home_elo)
    elo_list.append(away_elo)

elo_df = pd.DataFrame({"Team": team_list, "ELO": elo_list, "Date": date_list})


    

In [3]:
elo_df

,Team,ELO,Date
0,Valencia,25.000000,2006-08-26
1,Betis,25.000000,2006-08-26
2,Ath Bilbao,25.000000,2006-08-27
3,Sociedad,25.000000,2006-08-27
4,Espanol,25.000000,2006-08-27
...,...,...,...
10415,Leganes,22.458682,2020-08-03
10416,Betis,24.068836,2020-08-03
10417,Real Madrid,29.788006,2020-08-03
10418,Eibar,23.587169,2020-10-03


In [7]:
df.query('Season=="2011_2012" and HomeTeam=="Valencia" and AwayTeam=="Osasuna"')

,Division,Date,HomeTeam,AwayTeam,Full_Time_Result,Half_Time_Home_Team_Goals,Half_Time_Away_Team_Goals,Half_Time_Result,Home_Team_Shots,Away_Team_Shots,...,HOME_TRUESKILL_MU_NO_RESET,AWAY_TRUESKILL_MU_NO_RESET,HOME_TRUESKILL_SIGMA_NO_RESET,AWAY_TRUESKILL_SIGMA_NO_RESET,DRAW_CHANCE_NO_RESET,HOME_TRUESKILL_MU_SEASON,AWAY_TRUESKILL_MU_SEASON,HOME_TRUESKILL_SIGMA_SEASON,AWAY_TRUESKILL_SIGMA_SEASON,DRAW_CHANCE_SEASON
2258,SP1,2012-02-05,Valencia,Osasuna,1,0.0,0.0,D,23.0,11.0,...,26.656193,24.896224,0.77135,0.759257,0.942011,25.574719,24.973066,1.315602,1.256658,0.950958
